# Creating an MCP Server with FastMCP

## What is MCP (Model Context Protocol)?

MCP is a standardized protocol for LLMs to interact with external tools and data sources. It allows:
- **Tools**: Functions that LLMs can call to perform actions
- **Resources**: Data sources that LLMs can read and reference
- **Prompts**: Pre-defined prompts that can guide LLM behavior

## What is FastMCP?

FastMCP is a Python framework that simplifies creating MCP servers. It handles:
- Protocol communication automatically
- Type validation using Pydantic
- Easy tool/resource registration via decorators
- Built-in error handling

## Why Use MCP Servers?

- **Integration**: Connect LLMs to databases, APIs, or custom tools
- **Extensibility**: Add new capabilities without modifying the LLM
- **Standardization**: Use the same protocol across different AI applications
- **Control**: Manage what tools LLMs can access

## Simple MCP Server Example

Below is a basic MCP server with a few useful tools:
- **greet**: Greets a user by name
- **calculate**: Performs simple arithmetic
- **get_weather**: Returns mock weather data

In [1]:
from fastmcp import FastMCP
from typing import Optional

In [2]:
# Create an MCP server instance
mcp = FastMCP("example-server", "1.0.0")

In [3]:
# Define a tool using the @mcp.tool() decorator
@mcp.tool()
def greet(name: str, age: Optional[int] = None) -> str:
    """
    Greet a user by name with optional age information.
    
    Args:
        name: The name of the person to greet
        age: Optional age of the person
    
    Returns:
        A greeting message
    """
    if age:
        return f"Hello {name}! You are {age} years old."
    return f"Hello {name}!"

In [4]:
@mcp.tool()
def calculate(operation: str, a: float, b: float) -> float:
    """
    Perform a simple arithmetic operation.
    
    Args:
        operation: One of 'add', 'subtract', 'multiply', or 'divide'
        a: First number
        b: Second number
    
    Returns:
        The result of the operation
    """
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Cannot divide by zero")
        return a / b
    else:
        raise ValueError(f"Unknown operation: {operation}")


@mcp.tool()
def get_weather(city: str) -> dict:
    """
    Get mock weather data for a city.
    
    Args:
        city: The name of the city
    
    Returns:
        A dictionary with weather information
    """
    # Mock weather data
    weather_data = {
        "London": {"temperature": 12, "condition": "Cloudy", "humidity": 75},
        "New York": {"temperature": 5, "condition": "Snowy", "humidity": 60},
        "Tokyo": {"temperature": 15, "condition": "Partly Cloudy", "humidity": 70},
        "Sydney": {"temperature": 28, "condition": "Sunny", "humidity": 50},
    }
    
    if city in weather_data:
        return {"city": city, **weather_data[city]}
    else:
        return {"city": city, "error": "Weather data not available for this city"}


print("✓ MCP Server defined with 3 tools:")
print("  - greet(name, age)")
print("  - calculate(operation, a, b)")
print("  - get_weather(city)")

✓ MCP Server defined with 3 tools:
  - greet(name, age)
  - calculate(operation, a, b)
  - get_weather(city)


## Running the MCP Server

To run this server as a standalone MCP service, you would:

### 1. Create a Python script (`server.py`):
```python
from fastmcp import FastMCP
from typing import Optional

mcp = FastMCP("example-server", "1.0.0")

@mcp.tool()
def greet(name: str, age: Optional[int] = None) -> str:
    """Greet a user"""
    if age:
        return f"Hello {name}! You are {age} years old."
    return f"Hello {name}!"

# ... other tools ...

if __name__ == "__main__":
    mcp.run()
```

### 2. Run the server:
```bash
python server.py
```

### 3. Connect from an LLM client:
The server exposes its tools via the MCP protocol, which LLM clients (like Claude with MCP integration) can discover and use automatically.

## Key FastMCP Features

- **@mcp.tool()**: Decorator to register a function as a callable tool
- **Type hints**: Pydantic automatically validates input/output types
- **Docstrings**: Used for tool descriptions and documentation
- **Error handling**: Exceptions are automatically caught and formatted
- **Multiple tools**: Register as many tools as needed with decorators
- **Resources**: Can also define resources with `@mcp.resource()` for data access
- **Prompts**: Can define system prompts with `@mcp.prompt()`

## Testing the Server with a Client

A client script (`client.py`) has been created to test your MCP server. The client:

1. **Starts the server** as a subprocess
2. **Discovers available tools** using the `tools/list` RPC method
3. **Calls each tool** with different parameters
4. **Displays results** in a formatted way
5. **Stops the server** when done

### How to Run the Client

In your terminal, run:
```bash
python client.py
```

This will:
- Start your `server.py` in the background
- Send JSON-RPC requests to test all three tools
- Display formatted results for each test
- Automatically stop the server when done

### What the Client Tests

- **Tool Discovery**: Lists all available tools on the server
- **greet()**: Tests with just a name, and with name + age
- **calculate()**: Tests addition and multiplication operations
- **get_weather()**: Tests with known and unknown cities

### MCP Client Architecture

The client uses:
- **JSON-RPC 2.0**: Standard protocol for remote procedure calls
- **STDIO Transport**: Communicates via stdin/stdout with the server
- **Async/await**: For non-blocking I/O operations
- **Subprocess**: Spawns the server as a child process

# Understanding MCP: Complete Process Flow

## The Big Picture

An MCP (Model Context Protocol) workflow involves three main components:

```
┌─────────────────────────────────────────────────────────────────┐
│                    THE MCP ARCHITECTURE                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                   │
│  [LLM Client]  ←→  [MCP Protocol]  ←→  [MCP Server]             │
│  (requests)       (JSON-RPC 2.0)       (provides tools)          │
│                                                                   │
│  Examples:                             Examples:                 │
│  - Claude Desktop                      - Your server.py          │
│  - Custom Python Client                - Database query server   │
│  - Web Application                     - API wrapper server      │
│                                                                   │
└─────────────────────────────────────────────────────────────────┘
```

## Step-by-Step Process (What Happens When You Run `python client.py`)

### Step 1️⃣: Client Starts
```
python client.py
    ↓
MCPClient class instantiated
    ↓
Stores path to server.py ("server.py")
```

### Step 2️⃣: Client Spawns Server as Subprocess
```
client.start_server()
    ↓
subprocess.Popen([python, server.py])
    ↓
Server process starts with:
  - stdin: receives input from client
  - stdout: sends output to client
  - stderr: error logs
```

### Step 3️⃣: Server Initializes
```
server.py runs
    ↓
FastMCP("example-server", "1.0.0") creates server instance
    ↓
@mcp.tool() decorators register functions as callable tools:
  - greet()
  - calculate()
  - get_weather()
    ↓
mcp.run() starts listening on STDIO
    ↓
Server displays:
  [INFO] Starting MCP server 'example-server' with transport 'stdio'
  (Server is now waiting for incoming requests)
```

### Step 4️⃣: Client Discovers Available Tools
```
client.list_tools()
    ↓
Creates JSON-RPC request:
{
    "jsonrpc": "2.0",
    "id": 1,
    "method": "tools/list",
    "params": {}
}
    ↓
Sends to server via stdin
    ↓
Server receives request, processes it
    ↓
Server sends response via stdout with all available tools
    ↓
Client receives and parses response
```

### Step 5️⃣: Client Calls a Tool (Example: greet)
```
client.call_tool("greet", {"name": "Alice"})
    ↓
Creates JSON-RPC request:
{
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/call",
    "params": {
        "name": "greet",
        "arguments": {"name": "Alice"}
    }
}
    ↓
Sends to server via stdin
    ↓
Server receives request
    ↓
Validates arguments using type hints
    ↓
Calls Python function: greet("Alice")
    ↓
Function returns: "Hello Alice!"
    ↓
Server wraps result in JSON-RPC response:
{
    "jsonrpc": "2.0",
    "id": 2,
    "result": "Hello Alice!"
}
    ↓
Sends response via stdout
    ↓
Client receives, parses, and displays result
```

### Step 6️⃣: Repeat for Each Tool Call
```
Steps 5 repeat for:
  - greet("Bob", 30)
  - calculate("add", 15, 7)
  - calculate("multiply", 6, 9)
  - get_weather("London")
  - get_weather("Paris")
```

### Step 7️⃣: Client Stops Server
```
client.stop_server()
    ↓
process.terminate()
    ↓
Server shuts down gracefully
    ↓
Client exits
```

## Communication Protocol: JSON-RPC 2.0

MCP uses **JSON-RPC 2.0** - a lightweight remote procedure call protocol.

### Request Format
```json
{
    "jsonrpc": "2.0",           // Protocol version
    "id": 1,                     // Unique message ID (for matching responses)
    "method": "tools/list",      // What to do: list tools, call tool, etc.
    "params": {                  // Arguments to the method
        "name": "greet",
        "arguments": {"name": "Alice"}
    }
}
```

### Response Format
```json
{
    "jsonrpc": "2.0",           // Protocol version
    "id": 1,                     // Same ID as the request (links them together)
    "result": "Hello Alice!"     // The result data (or error)
}
```

### Or if there's an error:
```json
{
    "jsonrpc": "2.0",
    "id": 1,
    "error": {
        "code": -32600,
        "message": "Invalid Request"
    }
}
```

## Deep Dive: Inside Your Code

### What Happens in `server.py`

```
1. Import FastMCP
   ↓
2. Create FastMCP instance:
   mcp = FastMCP("example-server", "1.0.0")
   
3. Register tools using decorators:
   @mcp.tool()
   def greet(name: str, age: Optional[int] = None) -> str:
       ...
   
   FastMCP records:
   - Tool name: "greet"
   - Parameters: name (required), age (optional)
   - Type hints: str, Optional[int]
   - Docstring: Used as description
   
4. mcp.run()
   - Starts listening on STDIO
   - Waits for incoming JSON-RPC requests
   - When a request arrives:
     a. Parse JSON
     b. Validate tool name exists
     c. Validate arguments match type hints
     d. Call the Python function
     e. Serialize return value to JSON
     f. Send response back on STDOUT
```

### What Happens in `client.py`

```
1. Create MCPClient instance with path to server.py

2. start_server():
   - Uses subprocess.Popen() to start server.py
   - Connects to server's stdin/stdout
   
3. list_tools():
   - Build JSON request: {"method": "tools/list", ...}
   - Write to server's stdin
   - Read response from server's stdout
   - Parse and return tools list
   
4. call_tool(name, arguments):
   - Build JSON request with tool name and arguments
   - Write to server's stdin
   - Read response from server's stdout
   - Extract and return result or error
   
5. All communication happens via stdin/stdout:
   - Client writes JSON request + newline to stdin
   - Server reads from stdin, processes, writes to stdout
   - Client reads JSON response from stdout
```

## Real Example: greet("Alice")

Here's exactly what happens when the client calls `greet("Alice")`:

### On the Client Side (client.py):
```python
result = await self.call_tool("greet", {"name": "Alice"})
```

This creates a request:
```json
{
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/call",
    "params": {
        "name": "greet",
        "arguments": {"name": "Alice"}
    }
}
```

And writes it to stdin: `stdin.write(request_json + "\n")`

### On the Server Side (server.py):
```
1. FastMCP reads from stdin
2. Receives the JSON request
3. Parses it:
   - method = "tools/call"
   - name = "greet"
   - arguments = {"name": "Alice"}
   
4. Looks up tool "greet" (finds it)
5. Validates arguments:
   - "name" exists: ✓
   - "name" is string: ✓
   - age is not provided (optional): ✓
   
6. Calls the function:
   greet("Alice")
   
7. Function executes:
   if age:  # False, age not provided
       ...
   return f"Hello {name}!"  # "Hello Alice!"
   
8. Returns "Hello Alice!"
9. Wraps in JSON response:
{
    "jsonrpc": "2.0",
    "id": 2,
    "result": "Hello Alice!"
}
10. Writes to stdout
```

### Back on the Client Side:
```python
response_line = self.process.stdout.readline()
# Reads: {"jsonrpc": "2.0", "id": 2, "result": "Hello Alice!"}

response = json.loads(response_line)
# Parses JSON into Python dict

print(f"Result: {response['result']}")
# Prints: Result: Hello Alice!
```

## The Communication Timeline

```
TIME    CLIENT                          SERVER
────────────────────────────────────────────────────────
 0ms    Starts                         
        │
 1ms    ├─ spawn process ────────────→ server.py starts
        │                              │ FastMCP initializes
        │                              │ Waits on stdin
        │
 10ms   ├─ send: tools/list ────────→ reads request
        │                              processes
        │                              ←────── send tools
 12ms   ├─ receive tools list          │
        │                              │
 15ms   ├─ send: call greet ─────────→ reads request
        │                              validates
        │                              calls function
        │                              ←────── send result
 17ms   ├─ receive result              │
        │ "Hello Alice!"               │
        │                              │
 20ms   ├─ send: call calculate ──────→ reads request
        │                              ←────── send result
 22ms   ├─ receive result              │
        │ 22                           │
        │                              │
        ... (more tool calls) ...      
        │                              │
100ms   └─ terminate ─────────────────→ shutdown
        │                              
101ms   Client exits                  Server exits
```

## Key Concepts Explained

### 1. **STDIO Transport**
- **STDIN**: Standard input - how the CLIENT sends data to SERVER
- **STDOUT**: Standard output - how the SERVER sends data to CLIENT
- **STDERR**: Standard error - for logging/debugging
- This is the "pipe" connecting client and server processes

### 2. **subprocess.Popen()**
- Creates a new child process (your server)
- Each child process gets its own stdin/stdout/stderr
- The parent process (client) can communicate via these pipes
- When you terminate the parent, the child stops too

### 3. **JSON-RPC 2.0**
- A standard way to make remote function calls
- Request contains: method name + parameters
- Response contains: result (or error)
- ID field links requests to responses

### 4. **Type Validation (Pydantic)**
- FastMCP uses type hints to validate arguments
- If you pass wrong type, FastMCP rejects it
- Prevents bugs before functions execute

### 5. **Async/Await**
- Non-blocking I/O operations
- Client doesn't freeze while waiting for server
- Allows handling multiple operations efficiently

## Real-World Analogy

Think of it like a **phone call between a customer and a restaurant**:

```
CLIENT (Customer)          SERVER (Restaurant)
─────────────────────────────────────────────

Dials number              Answers phone
    ↓                          ↓
Waits for answer ←──────── Restaurant picks up
    ↓                          ↓
"I want a pizza"  ────────→ Listens to order
    ↓                          ↓
Waits for response ←──────── Kitchen prepares
    ↓                          ↓
Receives "Your pizza       Says "Pizza ready
is ready"                  in 30 mins"
    ↓                          ↓
"I also want a             Listens to new
salad"              ────────→ request
    ↓                          ↓
Waits for response ←──────── Prepares salad
    ↓                          ↓
Receives response          Says status
    ↓                          ↓
Hangs up            ────────→ Phone ends
    ↓                          ↓
Customer leaves         Restaurant closes
```

Similarly:
- Client connects to server (dials)
- Sends requests (orders)
- Server processes and responds
- Repeat for multiple requests
- Client disconnects (hangs up)

## Summary: The 3 Main Parts

### 1️⃣ **SERVER (server.py)**
```
┌─────────────────────────────────────────┐
│         MCP SERVER                      │
├─────────────────────────────────────────┤
│                                         │
│  Listens on STDIN                      │
│  Receives JSON-RPC requests            │
│  Validates arguments                   │
│  Calls registered functions            │
│  Returns results on STDOUT             │
│                                         │
│  Your tools:                           │
│  ✓ greet()                             │
│  ✓ calculate()                         │
│  ✓ get_weather()                       │
│                                         │
└─────────────────────────────────────────┘
```

### 2️⃣ **CLIENT (client.py)**
```
┌─────────────────────────────────────────┐
│         MCP CLIENT                      │
├─────────────────────────────────────────┤
│                                         │
│  Spawns server subprocess              │
│  Sends requests on STDOUT              │
│  Receives responses on STDIN           │
│  Parses JSON responses                 │
│  Displays results                      │
│                                         │
│  What it does:                         │
│  1. Discover available tools           │
│  2. Call greet("Alice")                │
│  3. Call greet("Bob", 30)              │
│  4. Call calculate("add", 15, 7)       │
│  5. Call calculate("multiply", 6, 9)   │
│  6. Call get_weather("London")         │
│  7. Call get_weather("Paris")          │
│                                         │
└─────────────────────────────────────────┘
```

### 3️⃣ **PROTOCOL (JSON-RPC 2.0)**
```
┌─────────────────────────────────────────┐
│      COMMUNICATION PROTOCOL             │
├─────────────────────────────────────────┤
│                                         │
│  Request:                              │
│  {                                     │
│    "jsonrpc": "2.0",                   │
│    "id": 1,                            │
│    "method": "tools/call",             │
│    "params": {...}                     │
│  }                                     │
│                                         │
│  Response:                             │
│  {                                     │
│    "jsonrpc": "2.0",                   │
│    "id": 1,                            │
│    "result": "success" or "error"      │
│  }                                     │
│                                         │
└─────────────────────────────────────────┘
```

## Flow Diagram

```
USER RUNS: python client.py
    │
    ▼
┌─────────────┐
│ client.py   │ start_server()
│ creates     ├────────────────┐
│ subprocess  │                │
└─────────────┘                ▼
                           ┌──────────┐
                           │server.py │
                           │ starts   │
                           └──────────┘
                                ▲
                                │ listening on STDIN
    ┌────────────────────────────┘
    │
    ├─ list_tools() ──────────────→ Send JSON-RPC request
    │                               ├─ Server processes
    │                               └─ Send response
    ├─ receive tools list ←─────────────────────────┐
    │
    ├─ call_tool("greet", ...) ───→ Send JSON-RPC request
    │                               ├─ Validate arguments
    │                               ├─ Call function
    │                               └─ Send result
    ├─ receive result ←──────────────────────────────┐
    │
    ├─ call_tool(...) ────────────→ ... (repeat for each tool)
    │
    ├─ receive result ←───────────────────────────────┐
    │
    └─ stop_server() ─────────────→ SERVER TERMINATES
                                    └─ Process ends

USER SEES: Formatted results for all tool calls
```

# What is a Standalone MCP Service?

## Definition

A **standalone MCP service** is an MCP server that runs **independently as its own process**, separate from the client that uses it. It can:
- Start and stop on its own
- Run in the background
- Be used by multiple clients
- Be deployed on different machines
- Continue running even if a client disconnects

## Standalone vs. Embedded

### ❌ NOT Standalone (Embedded in Client):
```
┌──────────────────────────────────┐
│    Client Application            │
│  ┌────────────────────────────┐  │
│  │  MCP Server (inside)       │  │
│  │  - greet()                 │  │
│  │  - calculate()             │  │
│  └────────────────────────────┘  │
└──────────────────────────────────┘

Server ONLY exists when client runs
Server CANNOT be reused by other clients
```

### ✅ STANDALONE (Independent Process):
```
Process 1: MCP Server (server.py)     Process 2: Client (client.py)
┌──────────────────────────┐          ┌────────────────────┐
│ Standalone MCP Service   │          │  Client App        │
│                          │ ←────────│  Connects to       │
│ - greet()                │  JSON    │  server via STDIO  │
│ - calculate()            │  RPC     │                    │
│ - get_weather()          │  2.0     │  Can disconnect    │
│                          │          │  Server keeps      │
│ Runs independently       │          │  running           │
│ Can receive multiple     │          └────────────────────┘
│ client connections       │
└──────────────────────────┘

Server runs in its OWN process
Multiple clients CAN connect
Server continues running after client exits
```

## Your Current Setup

Your `server.py` is a **standalone MCP service** because:

### ✅ It's Independent
```python
if __name__ == "__main__":
    mcp.run()  # Runs on its own, waits for requests
```
- Can be started in a terminal without any client
- Listens indefinitely for incoming requests
- Doesn't depend on a specific client

### ✅ It's Reusable
Multiple clients can connect to it:
```
Terminal 1:                    Terminal 2:               Terminal 3:
$ python server.py             $ python client.py        $ python client.py
(server running)               (client connected)        (client connected)
```

### ✅ It's Persistent
The server keeps running even when clients disconnect:
```
Timeline:
─────────────────────────────────────────────────────────
0ms    $ python server.py
       Server starts, listens on STDIO
       
10ms   $ python client.py (Terminal 2)
       Client connects
       Client calls tools
       Client disconnects
       
       ← Server is STILL RUNNING! ←
       
20ms   $ python client.py (Terminal 3)
       Another client connects
       This client can now use the server
       Client disconnects
       
       ← Server is STILL RUNNING! ←
       
100ms  Press Ctrl+C in Terminal 1
       Server stops
```

## How Your Current client.py Works vs. Standalone

### Your Current Approach (client.py spawns server):
```python
# client.py
def __init__(self, server_path: str):
    self.process = None

async def start_server(self):
    self.process = subprocess.Popen([sys.executable, self.server_path])
    # Starts server as subprocess

async def stop_server(self):
    self.process.terminate()
    # Stops server when done
```

**Result**: Temporary server (only exists while client runs)
```
Timeline:
Client starts → Server starts → Tools are called → Server stops
```

### True Standalone Approach (server runs separately):
```bash
# Terminal 1: Start standalone server
$ python server.py
[INFO] Starting MCP server 'example-server' with transport 'stdio'
(Server is now running, waiting for clients)

# Terminal 2: Client connects to existing server
$ python client_standalone.py
(Client connects, uses tools, disconnects)
(Server is STILL RUNNING)

# Terminal 3: Another client can connect
$ python client_standalone.py
(Another client connects, uses tools, disconnects)
(Server is STILL RUNNING)

# Back to Terminal 1: Stop the server manually
Ctrl+C
[INFO] Shutting down server
```

### Comparison Table

| Aspect | Your Current (Embedded) | Standalone |
|--------|--------------------------|------------|
| **Server starts** | When client starts | Manually or with system boot |
| **Server stops** | When client exits | When you stop it manually |
| **Multiple clients** | Not really (spawns new server each time) | Yes (one server, many clients) |
| **Deployment** | Client is self-contained | Server and clients separate |
| **Use case** | Testing, simple scripts | Production, shared services |
| **Persistence** | Server dies with client | Server runs independently |

## Real-World Analogy

### Non-Standalone (Your Current):
```
┌──────────────────────────────────┐
│    Restaurant (Client)           │
│                                  │
│    You: "I want a pizza"         │
│    │                             │
│    ├─ Creates a kitchen inside   │
│    │  the restaurant just now    │
│    │  ↓                          │
│    ├─ Kitchen cooks pizza        │
│    │  ↓                          │
│    ├─ Kitchen closes immediately │
│    │  after serving              │
│    │                             │
│    You get your pizza            │
│    Restaurant ends               │
└──────────────────────────────────┘
```

### Standalone:
```
┌──────────────────────────────────┐    ┌─────────────────────┐
│    Central Kitchen (Server)      │    │   Restaurant (Client)│
│                                  │    │                     │
│  Runs 24/7                       │←───→│ You order a pizza   │
│  Always ready to cook            │    │ Kitchen prepares    │
│  Serves many customers           │    │ You get pizza       │
│  Doesn't close when a customer   │    │ You leave           │
│  leaves                          │    │                     │
│                                  │    └─────────────────────┘
│  Another Customer (Another Client)
│                                  │    ┌─────────────────────┐
│  ────────────────────────────────→   │ Orders a burger     │
│  Kitchen prepares                │    │ Kitchen prepares    │
│  ←────────────────────────────────   │ Gets burger         │
│                                  │    │ Leaves              │
│  Kitchen is STILL RUNNING        │    │                     │
│                                  │    └─────────────────────┘
└──────────────────────────────────┘
```

## How to Use Your server.py as a True Standalone Service

### Option 1: Manual (What You Did)
```bash
# Terminal 1: Start the server
python server.py

# It will display:
# [INFO] Starting MCP server 'example-server' with transport 'stdio'
# (Server is now running indefinitely)

# Keep this terminal open!
```

Now you can create a simple client that connects to the running server:

```python
# simple_client.py - Connects to an already-running server
import subprocess
import json
import sys

# NOTE: This assumes server.py is already running in another terminal!
# It does NOT spawn the server.

# Start your own server process manually first
server = subprocess.Popen(
    [sys.executable, 'server.py'],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    text=True
)

# Send a request
request = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "tools/list",
    "params": {}
}

server.stdin.write(json.dumps(request) + "\n")
server.stdin.flush()

# Receive response
response = json.loads(server.stdout.readline())
print("Available tools:", response)
```

### Option 2: Using System Services (Production)
In a real production environment, you'd run the server as a background service:

```bash
# Linux/Mac: Run in background
nohup python server.py &

# Windows: Run as service or in background
start /B python server.py

# Docker: Package as container
docker run -d my-mcp-server:latest
```

The server would:
- Start automatically
- Keep running in the background
- Accept connections from any client
- Persist through system reboots (if configured)

## Key Characteristics of Standalone Services

| Characteristic | Description |
|---|---|
| **Independence** | Runs as separate process, not inside client |
| **Longevity** | Continues running even when no clients connected |
| **Reusability** | Multiple different clients can connect |
| **Discovery** | Clients need to know server's address/connection info |
| **Deployment** | Separate deployment from clients |
| **Management** | Needs startup/shutdown management |
| **Scalability** | Can handle multiple concurrent clients |

## Summary

```
STANDALONE MCP SERVICE = An MCP server that:
├─ Runs independently as its own process
├─ Can be started and stopped separately from clients
├─ Can accept requests from multiple different clients
├─ Continues running even when no client is connected
└─ Is deployed and managed as a separate service
```

Your `server.py` **is a standalone MCP service** because:
- ✅ It has `if __name__ == "__main__": mcp.run()`
- ✅ It listens for incoming requests indefinitely
- ✅ It doesn't need a specific client to run
- ✅ Multiple clients could theoretically connect to it

Your `client.py` is **different** because:
- ❌ It spawns its own server subprocess
- ❌ The server stops when the client finishes
- ❌ Not a true "standalone" usage